In [3]:
import urllib.request as urllib2
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import Request, urlopen
import datetime
import bs4
import re 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
%matplotlib inline  
pd.set_option('display.max_columns', None)
from time import sleep
os.environ["PATH"] ="C:\dev\chromedriver;C:\dev;" + os.environ['PATH']
import warnings
warnings.filterwarnings('ignore')

## Functions

In [142]:
def get_url(section):
    a_s=section.findAll("a")
    for a in a_s:
        if a.has_attr('href'):
            # the first link is the right one
            return a["href"]
        #TODO check link is valid

def validate_string(s):
    is_s=False
    if type(s)==str:
        if s!="":
            is_s =True
    return is_s

def find_vintage(content):
    result = ""
    # from 1985
    if "from " in content:
        split_on_from =  content.split("from ")
        if len(split_on_from)==2:
            #"guitar from 1985"
            year = split_on_from[1][:4]
            # still check that we are returning a number
            if year.isdigit():
                if check_vintage(year):
                    result = year
        else:
            # "guitar from the best from 1985"
            for i in range(len(split_on_from)):
                year = split_on_from[i][:4]
                # still check that we are returning a number
                if year.isdigit():
                    if check_vintage(year):
                        result = year
    else: 
        potential_years = []
        for i in range(len(content)):
            if len(content[i:])>4:
                four_char = content[i:i+4]
                if four_char.isdigit():
                    if check_vintage(four_char):
                        potential_years.append(four_char)
            if len(potential_years)==1:
                # "guitar blabla 1985"
                result=potential_years[0]
            elif len(potential_years)>1:
                # "guitar made in 1985, bought in 2001"
                result = min(potential_years)
    return result
        # TODO check for "made in", "manufactured in" etc

def find_longest_string_of_digits(s):
    i=0
    n = len(s)
    while s[i].isdigit() or (s[i]==".") or (s[i]==","):
        i+=1
        if (i==n):
            break
    return s[:i]

def find_price(content):
    result = ""
    ### TODO "price", "price is", "USD", "usd"
    if "price is " in content:
        split_on_price_is = content.split("price is ")
        if len(split_on_price_is)==2:
            # price is 300,45
            # find
            if check_price(split_on_price_is[1]):
                result = find_longest_string_of_digits(split_on_price_is[1])
        else:
            ps = []
            # price is 45.00. price is negotiable:
            for i in range(len(split_on_price_is)):
                print(split_on_price_is[i])
                longest_digit = find_longest_string_of_digits(split_on_price_is[i])
                if longest_digit!="":
                    if check_price(longest_digit):
                        ps.append(longest_digit)
            if len(ps)==1:
                result = ps[0]
            elif len(ps)>1:
                result = ps[0]
    for sign in ["usd", "USD", "$", "price", "Price"]:
        if sign in content:
            split_on_dollar_sign = content.split(sign)
            p = find_longest_string_of_digits(split_on_dollar_sign[1][find_ind_of_next_digit_in_string(split_on_dollar_sign[1]):])
            if p.isdigit():
                if check_price(p):
                    result = p
    return result

def check_price(p,lb=100,ub=5000):
    if type(p)==str:
        p = float(p.replace(",",""))
    return p>lb  and p <ub

def check_vintage(v):
    if type(v)==str:
        v=int(v)
    return v>1900 and v<2021

def find_ind_of_next_digit_in_string(s):
    for i in range(len(s)):
        if s[i].isdigit():
            return i
        
def get_brand_and_model(brand_models,s):
    """
    s: string, content to look into
    brand_models:dict, eg:
        {"brand1":["model1","model2"],
        "brand2":["model1","model2"],
        "brand3":["model1","model2"],}
        
    return: {"brand":["brand1","brand2"], "model":["model1","model2"]}
    """
    result={"brand":[],"model":[]}
    for brand in brand_models.keys():
        if brand in s:
            result["brand"].append(brand)
    if result["brand"]!=[]:
        for brand in result["brand"]:
            for model in brand_models[brand]:
                if model in s:
                    result["model"].append(model)
    return result

def get_brand_model_from_title_content(brand_models, title, content):
    brand=""
    model=""
    brand_model = get_brand_and_model(brand_models, title)
    if len(brand_model["brand"])==1:
        brand = brand_model["brand"][0]
    if len(brand_model["model"])==1:
        model = brand_model["model"][0]
    
    # if I did not manage to get brand from title, then get it from content
    if brand=="":
        brand_model = get_brand_and_model(brand_models, content)
        if len(brand_model["brand"])==1:
            brand = brand_model["brand"][0]
        # and get model as well
        if len(brand_model["model"])==1:
            model = brand_model["model"][0]    
    return brand, model
            
def find_color(s):
    for c in ["blue", "red", "yellow", "pink", "green", "purple", "white", "grey", "black", "beige", "orange"]:
        if c in s:
            return c
        
def get_wood(s):
    wood = ""
    if "neck wood" in s:
        split_on_neck_wood = s.split("neck wood")
        split_on_neck_wood
    return wood

## REDDIT

In [139]:
reddit_url = "https://www.reddit.com/r/Gear4Sale/?count=25&after=t3_kqnp3f"
path="C:/dev/niche/guitar/data/"

In [143]:
r = requests.get(
    'https://www.reddit.com/r/Gear4Sale/new.json',
    headers={'user-agent': 'Mozilla/5.0'})

all_posts =r.json()['data']['children']

# TODO get list of brands and models
brand_models={"Fender":["standard"]}

df_reddit=pd.DataFrame(columns=["title","content","url","price","vintage","color","wood"])#"brand","model"])
k=0

for post in all_posts:
    title = post["data"]["title"]
    if any(ext in title for ext in ["sell", "wts","Sell","WTS"]):
        title = post["data"]["title"]
        content = post["data"]["selftext"]
        vintage = find_vintage(title)
        if vintage =="":
            vintage = find_vintage(content)
        price = find_price(content)
        if price!="" and vintage!="":
            color = find_color(content)
            wood = get_wood(content)
            url = post["data"]["url"]
            new_row=[title,content,url,price, vintage,color,wood]
            if not ((df_reddit['title'] == post_title) & (df_reddit['content'] == post_content)).any():
                df_reddit.loc[k]=new_row
                k+=1
df_reddit

,title,content,url,price,vintage,color,wood
0,WTS/WTT Kiesel Vader V6 2017 Aquaburst Headles...,ImageURL: [https://imgur.com/a/hFR7shH](https:...,https://www.reddit.com/r/Gear4Sale/comments/kv...,1700,2017,red,
1,WTS Gibson Custom Shop 1959 Historic Les Paul ...,Title fix*\n\nhttps://www.photo-pick.com/onlin...,https://www.reddit.com/r/Gear4Sale/comments/ku...,4000,1959,red,
2,WTS 2020 Mu-Tron Microtron IV $200,"Excellent-near mint condition, velcro on back....",https://www.reddit.com/r/Gear4Sale/comments/ku...,200,2020,None,


## The Gear Page

In [131]:
tgp_base_url = "https://www.thegearpage.net"
tgp_url = "https://www.thegearpage.net/board/index.php?forums/guitar-emporium.22/"
path="C:/dev/niche/guitar/data/"

r = requests.get(
    tgp_url)
soup = bs4.BeautifulSoup(r.text, 'html.parser')



In [144]:
df_tgp=pd.DataFrame(columns=["title","content","url","price","vintage","color","wood"])#,"brand","model"])
k=0

for div in soup.findAll("div", {"class":"structItem-cell structItem-cell--main"}):
    if any(tag in div.text.upper() for tag in ["FS" or "FOS" or "WTS" or "SELL" or "SALE"]):
        title = div.findAll("a", {"class":""})[0].text
        post_url = tgp_base_url+div.findAll("a")[1]["href"]
        post_rep = requests.get(post_url)
        post_soup = bs4.BeautifulSoup(post_rep.text, 'html.parser')
        bbWrappers = post_soup.findAll("div", {"class":"bbWrapper"})
        content = bbWrappers[0].text
        
        vintage = find_vintage(title)
        if vintage =="":
            vintage = find_vintage(content)
        price = find_price(content)
        if price!="" and vintage!="":
            color = find_color(content)
            wood = get_wood(content)
            new_row=[title,content,post_url,price, vintage,color,wood]
            if not ((df_tgp['title'] == post_title) & (df_tgp['content'] == post_content)).any():
                df_tgp.loc[k]=new_row
                k+=1
df_tgp

,title,content,url,price,vintage,color,wood
0,Paul Reed Smith McCarty 58 - Fire Burst Red - ...,Reverb Link Click Here\n\n$2600 - Shipped\n\n2...,https://www.thegearpage.net/board/index.php?th...,2600,2010,None,
1,PRS SE Paul's Guitar,"Back up for sale, buyer backed out.\n\nPRS SE ...",https://www.thegearpage.net/board/index.php?th...,699,2020,None,
2,PRS SE 245,Paul Reed Smith SE 245\n\nGuitar is in excelle...,https://www.thegearpage.net/board/index.php?th...,599,2020,None,
3,Rare Paul Reed Smith EG3,Prefactory black on black collectors conditio...,https://www.thegearpage.net/board/index.php?th...,1600,1993,black,
4,2012 Fano Alt De Facto JM6 $2200,Thinking about selling or trading my Fano. It’...,https://www.thegearpage.net/board/index.php?th...,2200,2012,red,
5,1976 Alvarez Model 5022,Fantastic guitar (Martin copy) in what I'd cal...,https://www.thegearpage.net/board/index.php?th...,340,1976,None,
6,Whitfill S Style Hardtail Trans White Heavy Relic,"I just bought this beautiful Whitfill ""S"", the...",https://www.thegearpage.net/board/index.php?th...,2200,2017,red,
